<a href="https://colab.research.google.com/github/ersilia-os/ersilia-intro-workshop/blob/main/notebooks/m3_building_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train an AI model

This notebook contains the basic steps to train a classifier for bioactivity prediction.

It is prepared to run on Google Colaboratory, if you want to run it locally make sure to create a conda environment with Python 3.10 and install the packages indicated below.

_Remember that the ! sign indicates a bash command, to run it in the terminal simply copy the command without !_

# Supervised Machine Learning

Uses previously **labeled** data to train an algorithm (i.e the output is known). The algorithm learns if it is doing right by comparing the predicted vs the real output.

Simplified steps:
1. Data collection and processing.
2. Division of training data in Train and Test sets.
3. Use the train set to train the model.
4. Predict an output for the test set and compare the predicted vs real results.
5. Improve the model until we are satisfied with the performance on the test set

## Types of supervised ML models
### Classification
Classification problems are characterized by having categorical output (i.e: active, inactive), so the model tries to predict to which class the input belongs. It can include several classes, is not limited to a binary classification.

### Regression
Regression problems are characterized by continuous variables, where the model tries to predict the exact value of the input (i.e: IC50 of a specific compound)

## Evaluation of supervised ML models

### Classification metrics

We obtain the following data to evaluate the model:
- Y_pred: predictions on the test set
- Y_real: real outcome of the test set

**Accuracy:** number of correct predictions divided by the total number of predictions (TP/(lenY)). For example, if we have predicted correctly 5 out of 10 data points --> Accuracy = 50%

**Precision**: identification of only real positives (with a 100% precision, a model does not classify any negatives as positive) --> TP/(TP+FP)

**Recall**: identification of all positives (with a 100% recall, no positive is classified as negative, but some negatives might be included in the positives) --> TP / (TP+FN)

**Confusion matrix**: plots the real vs the predicted values in a table, to easily obtain the FP, TP, TN, FN values.


### Regression metrics
In a regression task, we obtain as error the difference between the predicted value and the real value (i.e: IC50real=0.1, IC50pred = 0.5 --> error of 0.4).


**Mean Absolute Error:** mean of the absolute values of errors.

**Mean Squared Error:** mean of the square error. By squaring, larger errors are contributing more and therefore the model punishes them.

**Root Mean Squared Error:** root of the mean of square error to simplify interpretation (by using MSE, we also square the units which makes them difficult to interpret).

**R-square**: coefficient of determination, the amount of variance explained by the model (from 0 to 1, the closer to 1, the better our model is)

## 1. Install the necessary packages

For this exercise, we will need the following packages:
- RdKit: provides basic chemoinformatics tools
- Pandas: deals with tabular data (like csv files)
- Matplotlib and seaborn: plotting tools to visualise results
- LazyQSAR: a simple package to build fast AI models developed by Ersilia. LazyQSAR automatically installs RdKit, Scikit-Learn, Ersilia Compound Embeddings, FLAML, Mordred and Numpy among others

Google Colab already provides Pandas, Matplotlib and Seaborn pre-installed, so you do not need to install them. If you run this notebook locally please make sure to use the `pip install` command on the Terminal to get the packages in your conda environment

**If you restart the notebook, you will need to reinstall the packages. In a local environment, you do NOT need to reinstall the packages, just activate the conda environment again**

In [ ]:
# Package installation
!pip install lazyqsar==0.3

## 2. Load your data

Next, we will use the pandas package to load the training data. At the bare minimmum, you should have a .csv file with two columns: SMILES and Activity of the molecule against the desired target.

In Google Colab, go to the left hand side of the screen and click on the `folder` icon. There, you will see a `Upload file` icon to add your files to the Cloud. You can also call them from Google Drive. These files will be deleted if the notebook is closed, so you will need to re-upload them every time.

If you are working locally, just make sure to input the right path to your data file.

In [ ]:
# @title File Name
filename = "chembl325_processed.csv" # @param {type:"string"}


In [ ]:
# Load data into a Pandas DataFrame (df)
# We will use the read_csv function from Pandas to automatically upload the csv file in the right format for Python
# We need first to import the package Pandas, which we abbreviate as pd

import pandas as pd

df = pd.read_csv(filename)

In [ ]:
# we can now observe how our dataframe looks like
df

In [ ]:
# and its shape
df.shape

In [ ]:
# Let's get the variables right!
# If the column name changes, you need to change these variables
# Remember Python strings are case-sensitive
SMILES = "SMILES"
EXP = "pIC50"
BIN = "bin"

## 3. Define a cut-off

Since we will create a classifier, we need to decide at which cut-off we consider our molecules `Active` or `Inactive`.

We can visualise our data with a histogram for example to make the decision, or we can ask the original data producers for their expert recommendation.

In [ ]:
# We import the plotting package Matplotlib, abbreviated as plt
import matplotlib.pyplot as plt

# We select the variable we want to plot in the histogram, in this case the activity values
x = df[EXP] #here you need to write the exact column name, respecting Caps
print(x)

In [ ]:
# We now plot the data using the histogram function
plt.hist(x)

In [ ]:
# Depending on our data, we might want to improve the plot for better visualization
# here, we clip the maximmum values to 20
plt.hist(x, range=(x.min(), 20))
plt.xlabel("IC50 in uM")
plt.ylabel("Number of mols")

To identify the best activity cut-off for your data, you need to:
- Understand the experimental set up: what kind of assay was done? what is the units of activity measured?
- Understand what is the end goal for the model: is it obtaining the most active compounds? Is it to filter our some non-actives to reduce the test number?
- Ask the experts! Talk to the people who ran the experiment, if possible

Once you have the cut-off, transform the activities to binary. Typically, by consensus:
- 1: Active (the compound perturbs our biological system)
- 0: Inactive (the compound does not affect our biolgical system)

In [ ]:
# @title Activity cut-off { display-mode: "form" }
# We identify a good cut-off for our data and assign the variable cutoff
cutoff = 7 # @param {type:"number"}

In [ ]:
# @title Direction of activity { display-mode: "form" }
# Are our actives lower than the cut-off, or higher than the cut-off?
direction = "higher" # @param {type:"string"}

In [ ]:
# Here, we will create a new column in our file with the binary activity

if direction == "lower":
    df[BIN] = [1 if x <= cutoff else 0 for x in df[EXP]]
elif direction == "higher":
    df[BIN] = [1 if x >= cutoff else 0 for x in df[EXP]]
else:
    print("no direction specified. Please select lower or higher direction for the activity cut-off")

In [ ]:
# Let's check the output
df

In [ ]:
# We can print how many actives / inactives we have in our dataset
print("Total molecules: ", len(df))
print("Active molecules: ", len(df[df[BIN]==1]))
print("Inactive molecules: ", len(df[df[BIN]==0]))
print("Frequency of Actives (%): ", len(df[df[BIN]==1])/len(df)*100)

## 4. Divide the data into Train and Test sets

To make sure our model will have good performance, we need to keep part of the data as a test set, to evaluate the model once trained.

Typically, it is good practice to reserve ~20% of the data as test set. Make sure that the balance of actives / inactives is maintained both in the train and the test sets

In [ ]:
# We will use this function to split the data
# The function uses the sklearn train_test_split built-in method
from sklearn.model_selection import train_test_split
import numpy as np

def random_split(df, size):
    indices = np.arange(len(df))
    X_train, X_test, y_train, y_test, i_train, i_test = train_test_split(df[SMILES], df[EXP], indices, test_size=size, stratify=df[BIN])
    train = df.iloc[i_train]
    test = df.iloc[i_test]
    return train, test

In [ ]:
train, test = random_split(df, 0.2)

In [ ]:
# Let's check the train set
train

In [ ]:
# Let's check the test set
test

In [ ]:
# We can make sure the active / inactive balance was preserved
train_balance = len(train[train[BIN]==1])/len(train)*100
test_balance = len(test[test[BIN]==1])/len(test)*100

print("The frequency of Actives in the train set is: ", train_balance)
print("The frequency of Actives in the test set is: ", test_balance)

In [ ]:
# We can also save the train and test sets in case we want to explore them manually
# To save files, we use the pandas method "to_csv". Make sure to write the right path to your file if you are working locally

train.to_csv("train_set.csv", index=False)
test.to_csv("test_set.csv", index=False)

## 5. Featurization of molecules

The next step is to convert the molecules in our train set to vectors so that we can pass them as input for the machine learning model.
We will first use the most used fingerprint, the Morgan Descriptors, which can be easily calculated using RdKit.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Create the molecules from the SMILES string
smiles = train[SMILES]  # Replace this with your own SMILES string
mols = [Chem.MolFromSmiles(smi) for smi in smiles]

# Generate Morgan fingerprints
radius = 3  # Specify the radius of the circular fingerprint
nBits = 2048
fps = [AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) for mol in mols]
morgan_fps_train = [np.array(list(fp.ToBitString())).astype(int) for fp in fps]

In [ ]:
#Let's check the outcome!
morgan_fps_train

_* We have individually created the descriptors so that we can check how they look like. Luckily, LazyQSAR provides a single funtion to convert the smiles to vectors and train an ML model, so we will use that feature_

## 6. Train an ML model

We have our input (morgan descriptors) and the endpoint, the activity in binary format (1 or 0) so we are ready to train a classifier!
We will use the built-in method in LazyQSAR

In [ ]:
import lazyqsar as lq

smiles_train = train[SMILES]
y_train = train[BIN]
model = lq.MorganBinaryClassifier(time_budget_sec=60, estimator_list=["rf", "lgbm", "xgboost"])
model.fit(smiles_train, y_train)

## 7. Evaluate the model performance

After fitting the model, we need to evaluate how good it works on the test set. For that, we will run the model to make predictions with the test data and compare it to the real values.

In [ ]:
# We predict the probabilities of the model for the smiles in the test set
# We can pass SMILES directly because our method converts them to Morgan fingerprints

smiles_test = test[SMILES]
y_hat = model.predict_proba(smiles_test)
y_hat

In [ ]:
# We get two results for each molecule, why?
# We only want to keep the probability of Active (1), the second column
y_hat = y_hat[:,1]
y_hat

### The ROC-AUC curve

In [ ]:
from sklearn.metrics import roc_curve, auc

# We need the real results, the activity of the test set
y_test = test[BIN]

# We use the sklearn package to calculate the roc_curve and plot it
fpr, tpr, _ = roc_curve(y_test, y_hat)
auroc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='#50285a', lw=2, label=f'ROC curve (AUC = {auroc:.2f})')
plt.plot([0, 1], [0, 1], color='#bee6b4', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

### The Contingency table

The contingency table tells us how many molecules will be correctly predicted and how many will be false positives or false negatives.

For that, we need to convert the continuous model output (probabilities) to a binary outcome (1 or 0) using the cut-off we decide between 0 and 1. Typically, the cut-off is assigned at 0.5.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

proba_cutoff = 0.5
y_hat_bin = [1 if x >= proba_cutoff else 0 for x in y_hat]


cf_matrix = confusion_matrix(y_test, y_hat_bin)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Inactive','Active'], fontsize=14)
ax.yaxis.set_ticklabels(['Inactive','Active'], fontsize=14)

### The Classification Report

Sklearn can automatically provide a report with the measures of precision and recall for each class

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_hat_bin))

## 7. Save the model
If we want to reuse our model, we need to save it so it can be applied to new data!

In [ ]:
model.save("model_morgan.joblib")

## 8. What could we do to improve model performace?
- Use more time for training
- Try another featurizer method
- ...

In this example, we will test and evaluate another method for featurizing, the Ersilia Compound Embeddings.

In [ ]:
# Train a model using LazyQSAR and evaluate its performance

import lazyqsar as lq

smiles_train = train[SMILES]
y_train = train[BIN]

model = lq.ErsiliaBinaryClassifier(time_budget_sec=60, estimator_list=["rf", "lgbm", "xgboost"])
model.fit(smiles_train, y_train)

In [ ]:
y_hat

In [ ]:
smiles_test = test[SMILES]

y_hat = model.predict_proba(smiles_test)[:,1]
# We need the real results, the activity of the test set
y_test = test[BIN]

# We use the sklearn package to calculate the roc_curve and plot it
fpr, tpr, _ = roc_curve(y_test, y_hat)
auroc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='#50285a', lw=2, label=f'ROC curve (AUC = {auroc:.2f})')
plt.plot([0, 1], [0, 1], color='#bee6b4', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
proba_cutoff = 0.5
y_hat_bin = [1 if x >= proba_cutoff else 0 for x in y_hat]

cf_matrix = confusion_matrix(y_test, y_hat_bin)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Inactive','Active'], fontsize=14)
ax.yaxis.set_ticklabels(['Inactive','Active'], fontsize=14)

In [ ]:
print(classification_report(y_test,y_hat_bin))

In [ ]:
model.save("model_eosce.joblib")

## 9. Save the final model

To ensure we use all possible data, at the end of the pipeline we will train the model with the whole dataset (train and test) and save it as the final model for deployment.

We need to choose which descriptor and parameters will work better for our case and build the model accordingly. We obtain that from the best train-test experiment.

In [ ]:
# train the model with the full data

# load the whole dataset

df = pd.read_csv(filename)

# binarize the activity if needed
SMILES = "SMILES"
EXP = "pIC50"
BIN = "bin"

direction = "lower"
cutoff = 2.5

if direction == "lower":
    df[BIN] = [1 if x <= cutoff else 0 for x in df[EXP]]
elif direction == "higher":
    df[BIN] = [1 if x >= cutoff else 0 for x in df[EXP]]
else:
    print("no direction specified. Please select lower or higher direction for the activity cut-off")

# train the selected model
import lazyqsar as lq
smiles_train = df[SMILES]
y_train = df[BIN]
model = lq.MorganBinaryClassifier(time_budget_sec=60, estimator_list=["rf", "lgbm", "xgboost"])
model.fit(smiles_train, y_train)
model.save("final_model.joblib")